In [217]:
import pandas as pd
import matplotlib as matplotlib
import matplotlib.pyplot as plt
import matplotlib.style as style
plt.style.use('fivethirtyeight')

import warnings
warnings.filterwarnings(action='ignore')

import numpy as np
import statsmodels
import statsmodels.api as sm
from statsmodels.tsa.stattools import adfuller
import seaborn as sns
from scipy import stats

#Facebook Prophet Package
from fbprophet import Prophet
from fbprophet.diagnostics import cross_validation


print('Imports Complete')



Imports Complete


Read in required dataframes, clean up date-time values in order to aggregate data

In [234]:
#Read in dataframe containing data for every census tract and corresponding community name
all_locations_df=pd.read_csv(r'C:\Users\brandin_hirsch\Desktop\trips_all_locations_communities_nov_march.csv',usecols=['time','census_tract','trip_count','community_name'])

#Group together the  by community name and trip start time
all_locations_df=pd.DataFrame(all_locations_df.groupby(['time','census_tract','community_name'])['trip_count'].sum())
all_locations_df.reset_index(inplace=True)

#Convert trip start time to date, take out the minutes,leave it as a pandas object for further group bys
all_locations_df['time']=pd.to_datetime(all_locations_df['time'],infer_datetime_format=True)

all_locations_df.sort_values(by='time',inplace=True)

all_locations_df['time']=all_locations_df['time'].dt.strftime('%Y-%m-%d %H')

all_locations_df=pd.DataFrame(all_locations_df.groupby(['time','census_tract','community_name'])['trip_count'].sum())

all_locations_df.reset_index(inplace=True)

all_locations_df.head(5)


,time,census_tract,community_name,trip_count
0,2018-11-01 00,17031010100,0,6
1,2018-11-01 00,17031010300,0,8
2,2018-11-01 00,17031010400,0,18
3,2018-11-01 00,17031010501,0,5
4,2018-11-01 00,17031010502,0,3


Subset the data by community

In [429]:
def subset_community(all_locations_df,community):
        
    census_tract_com=all_locations_df.copy()

    census_tract_com=census_tract_com[census_tract_com['community_name']==community]

    census_tract_com.drop(labels=['community_name'],axis=1,inplace=True)

    return(census_tract_com)
    
census_tract_com0=subset_community(all_locations_df,0)    

census_tract_com1=subset_community(all_locations_df,1)    

census_tract_com2=subset_community(all_locations_df,2)    

census_tract_com3=subset_community(all_locations_df,3)

#Create an array of every hour in the dataset, will be used to later solve for missing date values in areas with low rideshare volume
time_index=census_tract_com1.time.unique()


Define a function to loop through each census tract, create a 1 week forecast, and return the forecast for every tract

In [431]:
def census_tract_forecast(df,time_index):
    
    all_forecasts=[]
    
    tracts=df.census_tract.unique()
    
    time_series_full=pd.DataFrame({'ds':time_index})
    
    for census_tract in tracts:  
           
        #Save the census tract name
        census_tract_name=census_tract
        
        #Filter for the right census tract       
        time_series=df[df['census_tract']==census_tract]
        
        #Drop census tract column and change column names due to Prophet format requirements
        time_series.drop(labels='census_tract',axis=1,inplace=True)
    
        time_series.columns=['ds','y'] #time series should be a dataframe with two columns, time and values
        
        #Create full dataframe by filling in missing time values for areas that skip times due to no rides recorded at certain times
        
        time_series_merged=time_series_full.merge(time_series,how='left',on='ds')
        
        time_series_merged.fillna(0,inplace=True)
        
        time_series_merged=pd.DataFrame({'ds':time_series_merged.ds,'y':time_series_merged.iloc[:,1]})
                
        #Save actual values for forecasted week
        actual_volume=time_series_merged.iloc[len(time_series_merged)-168:,1]
    
        #Create initial train series
        time_series_train=time_series_merged.iloc[:(len(time_series_merged)-(168))]   
        
        #Instantiate new model (Prophet model variable must be cleared after each fit)
        my_model=[]
        
        my_model=Prophet()
            
        my_model.add_country_holidays(country_name='US')
        
    
        #Fit and Predict
        my_model.fit(time_series_train)
        
        future_dates = my_model.make_future_dataframe(periods=168, freq='1h')
    
        forecast = my_model.predict(future_dates)
        
    
        forecast_df=forecast[['yhat','ds']]
    
        forecast_df=forecast_df.iloc[len(forecast_df)-168:]
        
        #Create columns for census tract name and actual values
        forecast_df['census_tract']=census_tract
        
        forecast_df['actual_volume']=actual_volume.values
       
        
        forecast_df.columns=['forecasted_volume','date_time','census_tract','actual_volume']
        
        forecast_df.forecasted_volume.clip(lower=0,inplace=True)
        
        all_forecasts.append(forecast_df)
               
    return(pd.concat(all_forecasts))
    

Create 1 week forecast for every census tract

In [448]:
census_tract_com0_forecasts=census_tract_forecast(df=census_tract_com0,time_index=time_index)

census_tract_com1_forecasts=census_tract_forecast(df=census_tract_com1,time_index=time_index)

census_tract_com2_forecasts=census_tract_forecast(df=census_tract_com2,time_index=time_index)

census_tract_com3_forecasts=census_tract_forecast(df=census_tract_com3,time_index=time_index)


INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with ye

INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with ye

INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with ye

INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with ye

INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with ye

...

INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with ye

INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with ye

INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with ye

INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with ye

INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with ye

Export data to CSV for visualization in Tableau

In [449]:
#Read in position data for each census tract
pos_data=pd.read_csv(r'C:\Users\brandin_hirsch\Desktop\pos_data_v3.csv',usecols=['census_tract','Lat','Long'])



#Merge forecasts with position data and export to csv
census_tract_com0_forecasts=census_tract_com0_forecasts.merge(pos_data,how='left',on='census_tract')

census_tract_com0_forecasts.to_csv(r'C:\Users\brandin_hirsch\Desktop\prophet_forecasts_com0.csv')

census_tract_com1_forecasts=census_tract_com1_forecasts.merge(pos_data,how='left',on='census_tract')

census_tract_com1_forecasts.to_csv(r'C:\Users\brandin_hirsch\Desktop\prophet_forecasts_com1.csv')

census_tract_com2_forecasts=census_tract_com2_forecasts.merge(pos_data,how='left',on='census_tract')

census_tract_com2_forecasts.to_csv(r'C:\Users\brandin_hirsch\Desktop\prophet_forecasts_com2.csv')

census_tract_com3_forecasts=census_tract_com3_forecasts.merge(pos_data,how='left',on='census_tract')

census_tract_com3_forecasts.to_csv(r'C:\Users\brandin_hirsch\Desktop\prophet_forecasts_com3.csv')

#Union all forecasts and export to CSV
all_forecasts=pd.concat([census_tract_com0_forecasts,census_tract_com1_forecasts,census_tract_com2_forecasts,census_tract_com3_forecasts])

all_forecasts.to_csv(r'C:\Users\brandin_hirsch\Desktop\all_prophet_forecasts.csv')

all_forecasts_copy=all_forecasts.copy()

all_forecasts_copy['forecasted_volume'].values[all_forecasts_copy['forecasted_volume'].values < 1] = 0

all_forecasts_copy['forecasted_volume']=all_forecasts_copy.forecasted_volume.round(0)

all_forecasts_copy.to_csv(r'C:\Users\brandin_hirsch\Desktop\all_prophet_forecasts_V1.csv')